Code and (maybe) some thoughts on \< nlp in action \> ch3 and ch4 (partial)

In [ ]:
'''3.4 主题建模'''
#文档词频率
from nltk.tokenize import TreebankWordTokenizer
from Collections import Counter
from nlpia.data.loaders import kite_text, kite_history #注：python3.10可能会出现问题
tokenizer = TreebankWordTokenizer
kite_intro = kite_text.lower()
intro_tokens = tokenizer.tokenize(kite_intro)
kite_history = kite_history.lower()
history_tokens = tokenizer.tokenize(kite_history)
intro_total = len(intro_tokens)

#词频率
intro_tf = {}
intro_counts = Counter(intro_tokens)
intro_tf['kite'] = intro_counts['kite']/intro_total

#稀缺度
num_docs_containing_kite = 0
for doc in [intro_tokens, history_tokens]:
    if 'kite' in doc:
        num_docs_containing_kite += 1

intro_idf = {}
intro_idf['kite'] = num_doc / num_docs_containing_kite

#考虑文档内部词频率与包含该词文档数
intro_tfidf = {}
intro_tfidf['kite'] = intro_tf['kite'] * intro_idf['kite']

'''TF-IDF DEFINE
log_tf = log(term_occurences_in_doc) - log(num_terms_in_doc)
log_log_idf = log(log(total_num_docs)-log(num_docs_containing_term))
log_tf_idf = log_tf+log_idf'''


In [ ]:
'''3.4.2 相关度排序'''
document_tfidf_vector = []
for doc in docs:
    vec = copy.copy(zerovector)
    tokens = tokenizer.tokenizer(doc.lower())
    token_counts = Counter(tokens)

    for key, value in token_counts.items():
        docs_containing_key = 0
        for _doc in docs:
            if key in _doc:
                docs_containing_key += 1
        tf = value / len(lexicon)
        if docs_containing_key:
            idf = len(docs)/docs_containing_key
        else:
            idf = 0
        vec[key] = tf*idf

    document_tfidf_vector.append(vec)


    query = 'How long does it take to get to the store'
    query_vec = copy.copy(zero_vector)
    '''similar process to get query_vec'''
    cosine_sim(query_vec, document_tfidf_vector[0])

In [2]:
'''3.4.3 sklearn构建TF-IDF矩阵'''
from sklearn.feature_extraction.text import TfidfVectorizer
docs = ['a men wearing a shirt','crazy beach on the floor']
corpus = docs
vectorize = TfidfVectorizer(min_df=1)
model = vectorize.fit_transform(corpus)
print(model.todense().round(2))

[[0.   0.   0.   0.58 0.   0.58 0.   0.58]
 [0.45 0.45 0.45 0.   0.45 0.   0.45 0.  ]]


In [ ]:
'''4.2隐性语义分析,4.3奇异值分解'''
#based on SVD, implement LSA
from nlpia.book.examples.ch04_catdog_lsa_3x6x16 import word_topic_vectors
from nlpia.book.examples.ch04_catdog_lsa_sorted import lsa_models, prettify_tdm
import numpy as np
import pandas as pd

print(word_topic_vectors.T.round(1))

bow_svd, tfidf_svd = lsa_models() #svd分解
tdm = bow_svd['tdm']

U, s, Vt = np.linalg.svd(tdm)
pd.DataFrame(U, index=tdm.index).round(2) #U矩阵是主题-词矩阵，将词-文档向量转化为主题-文档向量


In [ ]:
'''4.4主成分分析'''
#基于PCA的短消息语义分析
from sklearn.decomposition import PCA
pca = PCA(n_components=16)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs) #文档-主题矩阵

column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(),tfidf.vocabulary_keys())))
weights = pd.DataFrame(pca.components_, columns=terms, index=['topic{}'.format(i) for i in range(16)])
deals = weights['! ; ] : ] half off free crazy deal only '.split()]
deals.T.sum() #得到不同主题关于交易的情感倾向（正/负）

#基于截断的SVD语义分析
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=16, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)

#基于LSA的垃圾分类
import numpy as np
svd_topic_vectors = (svd_topic_vectors.T/ np.linalg.norm(svd_topic_vectors, axis=1)).T
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1) #归一化并计算向量之间的距离

